<a href="https://colab.research.google.com/github/kili-technology/kili-python-sdk/blob/main/recipes/import_labels_from_shapefiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Labels from Shapefiles

This tutorial explains how to use `kili.append_labels_from_shapefiles` function in the Kili SDK to import geometries
from shapefile files and convert them to annotations in your Kili projects.

## Introduction

Shapefiles are a standard geospatial data format that stores the location, shape, and attributes of geographic features.
They are commonly used in Geographic Information Systems (GIS) to represent points, lines, and polygons.

The `append_labels_from_shapefiles` function automatically converts this spatial data into Kili annotations.


## Prerequisites

Before using this feature, make sure you have the following:

- A Kili project of type `IMAGE` or `GEOSPATIAL`
- One or more shapefile files (`.shp`)
- Knowledge of the coordinate system (EPSG code) of your shapefiles

## Installation

This feature requires additional dependencies. Install them using:

```bash
pip install 'kili[gis]'
```

_This command installs the necessary libraries such as pyproj which are used for geospatial data manipulation._


## Function Structure

The `append_labels_from_shapefiles` function takes the following parameters:

| Parameter          | Type                 | Description                                                |
|--------------------|----------------------|------------------------------------------------------------|
| `project_id`        | str                  | The Kili project identifier                                |
| `asset_external_id` | str                  | The external identifier of the asset to annotate           |
| `shapefile_paths`   | List[str]            | List of paths to shapefile files                           |
| `job_names`         | List[str]            | List of job names corresponding to each shapefile          |
| `category_names`    | List[str]            | List of category names corresponding to each shapefile     |
| `from_epsgs`        | Optional[List[int]]  | Optional list of source EPSG codes for each shapefile      |


## Supported Geometry Types

The function supports the following shapefile geometry types:

- Points (Type 1) - Converted to "marker" type annotations in Kili
- Polylines (Type 3) - Converted to "polyline" type annotations in Kili
- Polygons (Type 5) - Converted to "semantic" type (mask) annotations in Kili

## Basic Usage Example

Here's a simple usage example:

```python
from kili.client import Kili

# Initialize Kili client
kili = Kili(api_key="your_api_key")

# Import labels from shapefiles
kili.append_labels_from_shapefiles(
    project_id="your_project_id",
    asset_external_id="satellite_image.tif",
    shapefile_paths=["roads.shp", "buildings.shp", "water_points.shp"],
    job_names=["ROADS", "BUILDINGS", "HYDROLOGY"],
    category_names=["ROAD", "BUILDING", "WATER_POINT"],
    from_epsgs=[4326, 4326, 4326]  # All shapefiles are in WGS84 (EPSG:4326)
)
```

## Advanced Example with Different Coordinate Systems

If your shapefiles use different coordinate systems:

```python

from kili.client import Kili

kili = Kili(api_key="your_api_key")

kili.append_labels_from_shapefiles(
    project_id="your_project_id",
    asset_external_id="sentinel2_image.jp2",
    shapefile_paths=["observation_points.shp", "parcels.shp", "protected_areas.shp"],
    job_names=["OBSERVATIONS", "PARCELS", "ZONES"],
    category_names=["OBS_POINT", "AGRICULTURAL_PARCEL", "NATURE_RESERVE"],
    from_epsgs=[4326, 2154, 3857]  # WGS84, Lambert93, Web Mercator
)
```

In this example, each shapefile uses a different coordinate system. The function will automatically convert all
geometries to WGS84 (EPSG:4326) before importing them into Kili.


## Troubleshooting

### Problem: ImportError

```bash
ImportError: This function requires the 'gis' extra dependencies.
Install them with: pip install kili[gis] or pip install 'kili[gis]'
```

Solution: Install the GIS dependencies:

```bash
pip install 'kili[gis]'
```

### Problem: Incorrect Coordinates or Invisible Annotations

Possible solutions:

- Check that you have specified the correct EPSG code for each shapefile
- Make sure the image in Kili is correctly georeferenced
- Check if the image has geospatial metadata with:

```
asset = kili.assets(project_id="your_project_id", fields=["jsonContent"])[0]
print(asset['jsonContent'])
```

### Problem: Categories Not Found

Solution: Check that the category names exactly match those in your Kili ontology:

```python
project = kili.projects(project_id="your_project_id", fields=["jsonInterface"])[0]
print(project["jsonInterface"])
```


## Conclusion

The `append_labels_from_shapefiles` function greatly simplifies the import of geospatial data into Kili, allowing you
to easily convert your existing GIS data into annotations usable for machine learning and manual annotation.
